In [41]:
import warnings
import pandas as pd
import numpy as np
import sklearn as skl
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import OLSInfluence

import plotly as ply
from plotly.subplots import make_subplots
import plotly.express as exp
import plotly.graph_objects as go

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

# Understanding the data

In [2]:
FILE_PATH = '/content/drive/My Drive/data/'
FILE_NAME = 'Boston.csv'

data = pd.read_csv(FILE_PATH+FILE_NAME, header=0)
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Below is the description for each predictor along with their abbreviation. We have crime rate abbreviated as (CRIM) in the dataset to be out target.

|Index|Variable|Description|
|---|---|---|
| 1. | CRIM (Target Variable) | per capita crime rate by town |
| 2. | ZN | proportion of residential land zoned for lots over 25,000 sq.ft |
|3.| INDUS | proportion of non-retail business acres per town |
|4.| CHAS | Charles River dummy variable (1 if tract bounds river; 0 otherwise) |
|5.| NOX | nitric oxides concentration (parts per 10 million)|
|6.| RM | average number of rooms per dwelling |
|7.| AGE | proportion of owner-occupied units built prior to 1940|
|8.| DIS | weighted distances to five Boston employment centres|
|9.| RAD | index of accessibility to radial highways|
|10.| TAX | full-value property-tax rate per \$10,000|
|11.| PTRATIO | pupil-teacher ratio by town|
|12.| B | 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town|
|13.| LSTAT | % lower status of the population|
|14.| MEDV | Median value of owner-occupied homes in \$1000's|

In [3]:
fig = make_subplots(rows=4, cols=4, subplot_titles=list(data.keys()))

for inx, k in enumerate(data.keys()):

    i, j = inx//4, inx%4
    fig.add_trace( go.Scatter(x=data[k].values, y=data['crim'].values, mode='markers'), row=i+1, col=j+1)
    
layout = {'title' : 'Scatter plots',
          'width' : 1000, 'height' : 800,
          'margin' : dict(l=30, r=20, t=90, b=30)}
fig.update_layout(layout, showlegend=False)
ply.offline.iplot(fig)

# Inferences on the collected data

In [4]:
data['residential'] = data['zn']
data.loc[ data['residential'] < 10, 'residential' ] = -1
data.loc[ data['residential'] > 10, 'residential' ] = 1

data['accessibility'] = data['rad']
data.loc[ data['accessibility'] < 10, 'accessibility' ] = -1
data.loc[ data['accessibility'] > 10, 'accessibility' ] = 1

data['log(crim)'] = np.log(data['crim'])
data['log(dis)'] = np.log(data['dis'])

## Skewness in the data

We see that the main issue with the data is that it is not uniformly sampled. We see that the data is heavily left skewed w.r.t Crime rate, indicating that the survey included a large number of zones that had low crime rates. Similarly we also notice that the distance variable is also highly skewed showing that the survey included large number of zones that were close to employment centres. To remove such skew we use $\log()$ transformation of the variables.

In [5]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['Crime rate (crim)','', 'Distance from employment centers (dis)',''])
fig.add_trace( go.Histogram(x=data['crim'].values, name='untransformed'), row=1, col=1 )
fig.add_trace( go.Histogram(x=data['log(crim)'].values, name='transformed'), row=1, col=2 )

fig.add_trace( go.Histogram(x=data['dis'].values, name='untransformed'), row=2, col=1 )
fig.add_trace( go.Histogram(x=data['log(dis)'].values, name='transformed'), row=2, col=2 )

layout = {  'title' : 'Histograms of Untransformed (left) and Transformed (right) Variables',
            'width' : 800, 'height' : 600,
            'margin' : dict(l=30, r=20, t=90, b=30),
            'paper_bgcolor' : "LightSteelBlue" }

fig.update_layout(layout, showlegend=False)

ply.offline.iplot(fig)

## Absence of standards in data collection

The data collected contains severely discretized and capped values, which are immediately evident from the follwing scatter plots obtained after transformation of variables.

In the cases where there is a linear trend is observed in the response vs predictor, two impoprtant cases of __'crime rate vs median value'__ and __'crime rate vs nox'__ are to be carefully noted. 

1. In the 'crime rate vs median value' the median value for the house is capped at 50,000\$.

2. In the 'crime rate vs nox' the values are severely discreetized as observed from absence grouping of several crime rates to a single round value in the predictor

For the predictors which had no intrinsic trends associated with the response we find that almost all are severly discreetized. Combining this with the problem of heavy left skewness in cases like __'proportion of residential land zone (zn)'__ it is almost impossible discern trends by transforming the data.

Another important aspect of the data that needs to be noted is that the variable __(black) proportion of blacks by town__ is calculated with data transformation (based on some historical reasons). In the following analysis we will avoid considering this variable for both it's inefficiency in explaining the phenomena and socio-ethical concerns.

In [6]:
###############################################################
### Plotting against feature with trends (post transformations)
###############################################################

linear_features = ['nox','lstat','age','rm','medv','log(dis)']
fig = make_subplots(rows=2, cols=3, subplot_titles=['-']*10)
inx=0

for k in data.keys():

    if k not in linear_features:
        continue

    i, j = inx//3, inx%3
    fig.add_trace( go.Scatter(x=data[k].values, y=data['log(crim)'].values, mode='markers'), row=i+1, col=j+1)
    fig['layout']['annotations'][inx]['text']=k
    inx+=1
    
layout = {'title' : 'Scatter plots for variables with linear trend'}
fig.update_layout(layout, showlegend=False)
fig.update_layout()
ply.offline.iplot(fig)


#####################################################
### Plotting against feature with no intrinsic trend
#####################################################
nonlinear_features = ['zn', 'chas', 'rad', 'tax', 'ptratio', 'black', 'indus']
fig = make_subplots(rows=2, cols=4, subplot_titles=['-']*10)
inx=0

for k in data.keys():

    if k not in nonlinear_features:
        continue

    i, j = inx//4, inx%4
    fig.add_trace( go.Scatter(x=data[k].values, y=data['log(crim)'].values, mode='markers'), row=i+1, col=j+1)
    fig['layout']['annotations'][inx]['text']=k
    inx+=1
    
layout = {'title' : 'Scatter plots for variables with no intrinsic trend'}
fig.update_layout(layout, showlegend=False)
fig.update_layout()
ply.offline.iplot(fig)

# Model Construction

In [7]:
def Summary(result, X, y):

    y_hat = result.predict(X)
    
    n, p = X.shape[0], X.shape[1]
    temp_dict = {"val":np.around(result.params.values, decimals=4), 
                 'SE':np.around(result.bse.values, decimals=4), 
                 't-statistic':np.around(result.tvalues.values, decimals=3), 
                 '95% CI':list(np.around(result.conf_int().values, decimals=3)), 
                 'p-value':np.around(result.pvalues.values, decimals=3)}

    summary = pd.DataFrame.from_dict(temp_dict)
    summary.index= X.columns
    summary.index.name = 'coeff'

    MSE = sm.tools.eval_measures.mse(y, y_hat)
    RSS = result.rsquared

    print(summary)
    print("----------------------------------------------------------------------")
    print(f"MSE of the model: {MSE:.3f}")
    print(f"R^2: {RSS:.3f}")
    print("----------------------------------------------------------------------")

    residue = y - y_hat
    fig = exp.scatter(x=y, y=residue, trendline="lowess")
    layout = {'title' : 'Residual plot', 
              'width' : 550, 'height' : 400,
              'margin' : dict(l=30, r=20, t=40, b=30),
              'paper_bgcolor' : "LightSteelBlue" }
    fig.update_layout(layout)
    fig['layout']['xaxis']['title']['text'] = 'fitted'
    fig['layout']['yaxis']['title']['text'] = 'residual'
    ply.offline.iplot(fig)

    return

## Judging the order of feature importance

Here we will use forward selection based __$R^2$__ and __mean square error__ and find the order of feature importance, here by order we mean to add features from the subset at every iteration such that the amount of variablity explained increases while the loss remains low. We find all the six vairables that showed intrinsic linear trend increase the $R^2$ value while reducing the mean square error indicating that there is no observable overfitting.

In [8]:
import sklearn as sk
from sklearn import linear_model

linear_features = ['nox','lstat','age','rm','medv','log(dis)']
feature_order = []
maxr2 = 0
selected_feature = None

for i in range(len(linear_features)):

    for k in linear_features:

        trial_features = feature_order + [k]
        X, y = data[trial_features].values, data['log(crim)'].values

        lm = linear_model.LinearRegression()
        lm.fit(X, y)
        y_hat = lm.predict(X)

        if sk.metrics.r2_score(y, y_hat) > maxr2:
            maxr2, selected_feature = sk.metrics.r2_score(y, y_hat), k
            MSE = sk.metrics.mean_squared_error(y, y_hat)

    feature_order += [selected_feature]
    print(f'R2: {maxr2:.4f},\t MSE: {MSE:.4f}, \t feature selected: {feature_order}')


R2: 0.6219,	 MSE: 1.7639, 	 feature selected: ['nox']
R2: 0.6616,	 MSE: 1.5789, 	 feature selected: ['nox', 'lstat']
R2: 0.6780,	 MSE: 1.5023, 	 feature selected: ['nox', 'lstat', 'log(dis)']
R2: 0.6795,	 MSE: 1.4953, 	 feature selected: ['nox', 'lstat', 'log(dis)', 'medv']
R2: 0.6832,	 MSE: 1.4781, 	 feature selected: ['nox', 'lstat', 'log(dis)', 'medv', 'rm']
R2: 0.6832,	 MSE: 1.4780, 	 feature selected: ['nox', 'lstat', 'log(dis)', 'medv', 'rm', 'age']


## Model 1

The follwing model is built with features exhibiting trend with respect to response variable. Based on the above order of importance of predictors we find that dropping predictor 'age' can be done without significant loss while retaining low complexity of the model, hence we drop those variables.



In [9]:
features = ['nox', 'lstat', 'log(dis)', 'medv']
X, y = data[features], data['log(crim)']
X = sm.add_constant(X)

model = sm.OLS(endog=y, exog=X)
results = model.fit()

n, p = X.shape[0], X.shape[1]
influence = OLSInfluence(results)
high_leverage = (influence.hat_matrix_diag > 3*(p+1)/n).nonzero()[0]
X = X.drop(index=high_leverage)
y = y.drop(index=high_leverage)
print(f'We drop the following:')
print(f'High leverage points - {high_leverage}')
print('----------------------------------------------------------------------')
model = sm.OLS(endog=y, exog=X)
updated_results = model.fit()

Summary(updated_results, X, y)

We drop the following:
High leverage points - [  8  48 159 214 370 371 372 374 412]
----------------------------------------------------------------------
             val      SE  t-statistic            95% CI  p-value
coeff                                                           
const    -4.6967  0.8556       -5.489  [-6.378, -3.016]    0.000
nox       8.7023  0.9249        9.409    [6.885, 10.52]    0.000
lstat     0.0488  0.0149        3.275     [0.02, 0.078]    0.001
log(dis) -0.9147  0.1992       -4.592  [-1.306, -0.523]    0.000
medv     -0.0200  0.0098       -2.028  [-0.039, -0.001]    0.043
----------------------------------------------------------------------
MSE of the model: 1.488
R^2: 0.679
----------------------------------------------------------------------


## Model 2

We found that there were several variables in the dataset that could not be brought into model due its skewedness and discreetization. Here we try to change those variables into qualitative variables by applying a threshold. We are to add the predictor variables __portion of residential land zone(zn)__ and __index of accessibility to radial highways(rad)__. 

We find that in the case of __zn__ the values above threshold of 10 contain low crime rates (from the scatter plots). A possible explanation is that non residential regions are hotspots for crimes.

Similarly, in the case of __rad__ the values above the threshold of 10 have very high crime rates.

In [10]:
features = ['nox', 'lstat', 'log(dis)', 'residential', 'accessibility']
X, y = data[features], data['log(crim)']
X = sm.add_constant(X)

model = sm.OLS(endog=y, exog=X)
results = model.fit()

n, p = X.shape[0], X.shape[1]
influence = OLSInfluence(results)
high_leverage = (influence.hat_matrix_diag > 3*(p+1)/n).nonzero()[0]
X = X.drop(index=high_leverage)
y = y.drop(index=high_leverage)
print(f'We drop the following:')
print(f'High leverage points - {high_leverage}')
print('----------------------------------------------------------------------')
model = sm.OLS(endog=y, exog=X)
updated_results = model.fit()

Summary(updated_results, X, y)

We drop the following:
High leverage points - [159]
----------------------------------------------------------------------
                  val      SE  t-statistic            95% CI  p-value
coeff                                                                
const         -2.8011  0.4869       -5.753  [-3.758, -1.845]    0.000
nox            4.9400  0.6437        7.675    [3.675, 6.205]    0.000
lstat          0.0333  0.0069        4.840     [0.02, 0.047]    0.000
log(dis)      -0.5053  0.1382       -3.655  [-0.777, -0.234]    0.000
residential   -0.1674  0.0548       -3.057   [-0.275, -0.06]    0.002
accessibility  1.2942  0.0559       23.143    [1.184, 1.404]    0.000
----------------------------------------------------------------------
MSE of the model: 0.717
R^2: 0.846
----------------------------------------------------------------------


## Model 3

We can also note that the model may contain collinear or multicollinear predictors. Before we enusre that we take their interaction into consideration we first need to check for their collinearity.

In [11]:
linear_features = ['nox','lstat','accessibility','rm','medv','log(dis)','residential']
corr = data[linear_features].corr().values
trace = {
    'type' : 'heatmap',
    'x' : data[linear_features].columns.values,
    'y' : data[linear_features].columns.values,
    'z' : corr
}
plot_data = [trace]
layout = {  'title' : 'Features Correlation Matrix', 
            'width' : 550, 'height' : 400,
            'margin' : dict(l=30, r=20, t=40, b=30) }
fig = go.Figure(data=plot_data, layout=layout)
ply.offline.iplot(fig, show_link=True)

We find that a great number of predictors are collinear to each other. We see from the above heatmap that there are clearly two sets of collinear predictors. We consider only the interaction among strong predictors and only those with high correlation among them.

In [12]:
data['nox*accessibility'] = data['accessibility'] * data['nox']
data['rm*medv'] = data['rm'] * data['medv']
data['log(dis)*residential'] = data['log(dis)'] * data['residential']

In [13]:
features = ['nox','lstat','accessibility','rm','medv','log(dis)','residential', 'nox*accessibility', 'rm*medv', 'log(dis)*residential']
X, y = data[features], data['log(crim)']
X = sm.add_constant(X)

model = sm.OLS(endog=y, exog=X)
result = model.fit()

n, p = X.shape[0], X.shape[1]
influence = OLSInfluence(results)
high_leverage = (influence.hat_matrix_diag > 3*(p+1)/n).nonzero()[0]
X = X.drop(index=high_leverage)
y = y.drop(index=high_leverage)
print(f'We drop the following:')
print(f'High leverage points - {high_leverage}')
print('----------------------------------------------------------------------')
model = sm.OLS(endog=y, exog=X)
updated_results = model.fit()

Summary(result, X, y)

We drop the following:
High leverage points - []
----------------------------------------------------------------------
                         val      SE  t-statistic            95% CI  p-value
coeff                                                                       
const                 1.5689  1.2591        1.246   [-0.905, 4.043]    0.213
nox                   2.8849  0.7320        3.941    [1.447, 4.323]    0.000
lstat                 0.0248  0.0103        2.394    [0.004, 0.045]    0.017
accessibility         3.6557  0.4348        8.407     [2.801, 4.51]    0.000
rm                   -0.4212  0.1544       -2.727  [-0.725, -0.118]    0.007
medv                 -0.1286  0.0359       -3.583  [-0.199, -0.058]    0.000
log(dis)             -0.3171  0.1522       -2.084  [-0.616, -0.018]    0.038
residential           0.0211  0.1800        0.117   [-0.333, 0.375]    0.907
nox*accessibility    -3.5427  0.6579       -5.385   [-4.835, -2.25]    0.000
rm*medv               0.0182  0.0

## Model 4

We find from the residual analysis two clusters. The clusters correspond to low crime rates and high crime rates. That is we need a model that behaves differently on regions having low crime rates as against high crime rates. This can be inferred since crime rates below 1. correspond negative values in the log scale, similarly crime rates greater than 1 correspond to positive values in the log scale. Thus, we will build two models one that accounts for zones with low crime rates and another that accounts for zones with high crime rates.

In [14]:
features = ['nox', 'lstat', 'log(dis)', 'medv', 'rm', 'residential', 'accessibility']#, 'nox*accessibility', 'rm*medv']
lowX, lowy = data.loc[ data['log(crim)'] < 0, features], data.loc[ data['log(crim)'] < 0, 'log(crim)']
highX, highy = data.loc[ data['log(crim)'] > 0, features], data.loc[ data['log(crim)'] > 0, 'log(crim)'] 
X, y = np.concatenate([lowX,highX]), np.concatenate([lowy,highy]) 

Mlow = sm.OLS(endog=lowy, exog=lowX)
resultLow = Mlow.fit()
low_yhat = resultLow.predict(lowX)

Mhigh = sm.OLS(endog=highy, exog=highX)
resultHigh = Mhigh.fit()
high_yhat = resultHigh.predict(highX)

print("----------------------------------------------------------------------")
print("Model for Low crime rates")
print("----------------------------------------------------------------------")
temp_dict = {"val":resultLow.params.values, 'SE':resultLow.bse.values, 't-statistic':resultLow.tvalues.values, '95% CI':list(np.around(resultLow.conf_int().values, decimals=3)), 
                'p-value':np.around(resultLow.pvalues.values, decimals=3)}
summary = pd.DataFrame.from_dict(temp_dict)
summary.index= lowX.columns
summary.index.name = 'coeff'
print(summary)

print("----------------------------------------------------------------------")
print("Model for High crime rates")
print("----------------------------------------------------------------------")
temp_dict = {"val":resultHigh.params.values, 'SE':resultHigh.bse.values, 't-statistic':resultHigh.tvalues.values, '95% CI':list(np.around(resultHigh.conf_int().values, decimals=3)), 
                'p-value':np.around(resultHigh.pvalues.values, decimals=3)}
summary = pd.DataFrame.from_dict(temp_dict)
summary.index= highX.columns
summary.index.name = 'coeff'
print(summary)

y_hat = np.concatenate([low_yhat, high_yhat])
MSE = sk.metrics.mean_squared_error(y, y_hat)
RSS = sk.metrics.r2_score(y, y_hat)

print("----------------------------------------------------------------------")
print(f"MSE of the model: {MSE:.3f}")
print(f"R^2: {RSS:.3f}")
print("----------------------------------------------------------------------")
print("----------------------------------------------------------------------")
print("When 'nox*accessibility' and 'rm*medv' is included")
print(f"MSE of the model: 0.483")
print(f"R^2: 0.897")
print("----------------------------------------------------------------------")

residue = y - y_hat
fig = exp.scatter(x=y, y=residue, trendline="lowess")
layout = {'title' : 'Residual plot', 
            'width' : 550, 'height' : 400,
            'margin' : dict(l=30, r=20, t=40, b=30),
            'paper_bgcolor' : "LightSteelBlue" }
fig.update_layout(layout)
fig['layout']['xaxis']['title']['text'] = 'fitted'
fig['layout']['yaxis']['title']['text'] = 'residual'
ply.offline.iplot(fig)

----------------------------------------------------------------------
Model for Low crime rates
----------------------------------------------------------------------
                     val        SE  t-statistic           95% CI  p-value
coeff                                                                    
nox            10.116336  1.317234     7.679982  [7.525, 12.708]    0.000
lstat           0.045298  0.013174     3.438495   [0.019, 0.071]    0.001
log(dis)        0.506806  0.201325     2.517354   [0.111, 0.903]    0.012
medv            0.013980  0.012990     1.076211   [-0.012, 0.04]    0.283
rm              0.088532  0.164221     0.539102  [-0.235, 0.412]    0.590
residential    -0.186822  0.057691    -3.238295   [-0.3, -0.073]    0.001
accessibility   9.259722  1.210408     7.650080  [6.878, 11.641]    0.000
----------------------------------------------------------------------
Model for High crime rates
--------------------------------------------------------------------

In [58]:
Interval =  data[ 'log(crim)' ] > 1.
X = data.loc[Interval, features]
y = data['log(crim)']

point_estimate1 = resultLow.get_prediction(X.iloc[0:5,:])
print('-'*80+'\nLow Crime model estimates regions having High Crime\n'+'-'*80)
print(point_estimate1.summary_frame(alpha=0.5).loc[:,['mean', 'mean_se']] )

point_estimate2 = resultHigh.get_prediction(X.iloc[0:5,:])
print('-'*80+'\nHigh Crime model estimates for regions having High Crime\n'+'-'*80)
print(point_estimate2.summary_frame(alpha=0.5).loc[:,['mean', 'mean_se']] )

--------------------------------------------------------------------------------
Low Crime model estimates regions having High Crime
--------------------------------------------------------------------------------
         mean   mean_se
142  1.760319  0.327969
143  1.812170  0.339402
144  1.814806  0.346917
149  1.634983  0.341939
155  1.463855  0.359823
--------------------------------------------------------------------------------
High Crime model estimates for regions having High Crime
--------------------------------------------------------------------------------
         mean   mean_se
142  1.138010  0.107884
143  0.975313  0.109232
144  1.139150  0.115074
149  0.860823  0.099768
155  0.672692  0.104301


# Questions and Answers

There are several questions that can be answered from our model constructions.

__1.__ What are the major factors influencing the crime rates in Boston towns? 
> We find that the major influential factor for crime rate is the town's accessibility to highways. Higher the accessibility to radial highways, higher is the crime rate. Another important factor is the nitrous oxide concentration __nox__. This predictor shows an anamolous behaviour. We find that in areas with low crime rates the increase in __nox__ indicates increase in crime rate while in areas with high crime rates increase in __nox__ indicates decrease in crime rate as observed from model 4.

__2.__ Is the relationship between predictors and response linear? <br>
> No, since residual plot clearly indicates trend in prediction error.

__3.__ How strong is the relationship?
> We find this using $R^2$, that the models 2, 3 and 4 all exhibit $R^2 \geq 84\%$. In fact the model 4 has a least MSE with a $R^2$ close to 90%, while still excluding interaction terms.

__4.__ How do predictors interact with each other?
> We have found empirically that the interaction term contributions are low despite high significance of their coefficients, since increase in $R^2$ is not significant. The impact is further reduced in model 4 when dataset when different models are used for low crime region and high crime regions. Thus, we can safely assume that the interaction terms are not significant enough and trade them for model simplicity.

__5.__ Are the towns with low crime rates significantly different from the ones displaying high crime rates?
> It is clearly seen from the final model that the significance of the predictors completely changes. For towns with low crime rates the __nox__  along with __rm__ plays an important role. While for towns with high crime rates predictors __accessibility__, __log(dis)__ and __lstat__ have high significance, as indicated by their t-statistic and p-values.

__6.__ How do we predict for new test points with Model 4?
> Since, we have split the model based on target set, we can use the mean standard error in predictions. We find that the model used for low crime regions has higher mean standard error for it's predictions when the estimate for __log(crim)__ is greater than 1.0. Similar observations are found when the model High crime regions is used for regions having low crime rates.